# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

## Setup

In [1]:
import sys
sys.path.append('..')  # Adjust the path as necessary to point to the root of your workspace
from utils import get_openai_api_key, get_openai_api_model, get_openai_api_endpoint

OPENAI_API_KEY = get_openai_api_key()
OPENAI_API_MODEL = get_openai_api_model()
OPENAI_API_ENDPOINT = get_openai_api_endpoint()
llm_config = [{"model": OPENAI_API_MODEL,
               "api_type": "azure",
               "temperature": 0.9,
               "api_key": OPENAI_API_KEY,
               "base_url": OPENAI_API_ENDPOINT,
               "api_version": "2024-02-15-preview"}] 



## Define an AutoGen agent

In [2]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

In [3]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!


In [4]:
reply = agent.generate_reply(
    messages=[{"content": "Make a new joke about donuts", "role": "user"}]
)
print(reply)

Why did the donut go to the dentist?

Because it had a cavity and wanted to get a filling!


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

In [6]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Well, hello there, Joe! I'm Cathy, and I'm here to tickle your funny bone. Let's keep those laughs coming! 

You know, being a stand-up comedian is a lot like being a preschool teacher. I'm constantly trying to engage an audience who has the attention span of a goldfish. And let's be honest, most of the time I'm just hoping nobody throws a tantrum! 

But hey, don't worry, Joe. I promise I won't put you in a timeout if you don't laugh at my jokes. Although, a timeout might give you some time to think about your life choices... starting with your questionable taste in stand-up comedy! 

You know, my friends always tell me I have a unique sense of humor. I think they're just trying to be polite and say, "Cathy, we have no idea what you're talking about, but we love you anyway." And honestly, I'm okay with that. 

So let's dive in

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

In [ ]:
pprint.pprint(chat_result.cost)

In [ ]:
pprint.pprint(chat_result.summary)

## Get a better summary of the conversation

In [ ]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
pprint.pprint(chat_result.summary)

## Chat Termination

Chat can be terminated using a termination conditions.

In [29]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [ ]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling. We end the conversation when we feel like it.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

In [ ]:
pprint.pprint(chat_result.summary)

## Debate Scenario

In [ ]:
cathy = ConversableAgent(
      "cathy",
      system_message="Your name is Cathy and you are to debate about whether Jose Rizal should be the national hero of the Philippines." 
      "You will represent the side that is against him being the hero. You will refute the points made by Joe and provide your own arguments in every turn.",
      llm_config={"config_list": llm_config},
      human_input_mode="NEVER",  # Never ask for human input.
   )

joe = ConversableAgent(
      "joe",
      system_message="Your name is Joe and you are to debate about whether Jose Rizal should be the national hero of the Philippines." 
      "You will represent the side that is for him being the hero. You will refute the points made by Cathy and provide your own arguments in every turn.",
      llm_config={"config_list": llm_config},
      human_input_mode="NEVER",  # Never ask for human input.
   )

result = joe.initiate_chat(
    cathy, 
    message="Cathy, let us start the debate.",
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation", 
    max_turns=6)

In [ ]:
pprint.pprint(result.summary)